In [ ]:
import pandas as pd
import numpy as np
import rtsvg
rt = rtsvg.RACETrack()
import time
import os
from ollama import chat
from ollama import ChatResponse
from pydantic import BaseModel
model = 'deepseek-r1:32b'
def promptModel(prompt):
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content': prompt,},])
    return response['message']['content']
_results_ = promptModel('What is 55*3?  Return a single number.') # force the model load
import random
import copy
def randomizeOrder(_lines_):
    _lines_ = copy.deepcopy(_lines_)
    new_lines = []
    while len(_lines_) > 0:
        _index_ = random.randint(0, len(_lines_) - 1)
        new_lines.append(_lines_.pop(_index_))
    return new_lines
social_media_lines = []
_mc3_dir_ = '../../../data/2014_vast/MC3'
_files_   = ['csv-1700-1830.csv','csv-1831-2000.csv','csv-2001-2131.csv']
for _file_ in _files_:
    _file_ = _mc3_dir_ + '/' + _file_
    with open(_file_, encoding='iso-8859-1') as f:
        _lines_ = f.readlines()
        social_media_lines.extend(_lines_)
newman_lines, non_newman_lines = [], []
for i in range(len(social_media_lines)):
    if 'newman' in social_media_lines[i].lower(): 
        if 'rally' in social_media_lines[i].lower(): 
            newman_lines.append(social_media_lines[i])
    else:                                         non_newman_lines.append(social_media_lines[i])
print(f'{len(social_media_lines)=} {len(newman_lines)=} {len(non_newman_lines)=}')

In [ ]:
model = 'phi4-mini'
def createNeedlePrompt(wo_newman, w_newman):
    _lines_  = []
    _prompt_ = f'Which doctor spoke at the POK Rally?  Use the following social media to answer the question.  Respond with their complete name and title in JSON format.  If there is no answer, respond with "None".\n\n'
    for i in range(w_newman):   _lines_.append(random.choice(newman_lines))
    for i in range(wo_newman):  _lines_.append(random.choice(non_newman_lines))
    _lines_ = randomizeOrder(_lines_)
    return _prompt_ + ''.join(_lines_)

class RallySpeaker(BaseModel):
    name: str

_lu_ = {'time_taken':[], 'name':[], 'valid':[], 'wo_newman':[], 'w_newman':[]}

for i in range(10, len(newman_lines)+len(non_newman_lines), 50):
    for j in range(0, 3): # Inclusions of a Newman Line
        for k in range(1): # Samples
            _prompt_ = createNeedlePrompt(i, j)
            t0 = time.time()
            response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content':  _prompt_,},], format=RallySpeaker.model_json_schema())
            t1 = time.time()
            rally_speaker = RallySpeaker.model_validate_json(response['message']['content'])
            _lu_['time_taken'].append(t1 - t0)
            _lu_['name'].append(rally_speaker.name)
            _lu_['wo_newman'].append(i)
            _lu_['w_newman'].append(j)
            if    j == 0: _lu_['valid'].append(rally_speaker.name.lower() == 'none')
            else:         _lu_['valid'].append('newman' in rally_speaker.name.lower())
            print('.', end='')

In [ ]:
#df_niah = pd.DataFrame(_lu_)
#df_niah.to_parquet(_mc3_dir_ + '/niah_4_phi4.parquet')
#rt.smallMultiples(df=df_niah, sm_type='histogram', category_by='w_newman', sm_params={'bin_by':'name'}, color_by='valid', w=1536)